In [4]:
# load libraries and dependencies
%pip install xlrd
%pip install openpyxl
import numpy as np
import pickle
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.svm import LinearSVC
#from sklearn import svm 
from sklearn.calibration import CalibratedClassifierCV
from nltk.corpus import stopwords
import time
from nltk.stem import WordNetLemmatizer
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from datetime import datetime
from joblib import dump, load
import json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# load train data created from LoadData_TestTrainSplit.ipynb
df_train = pd.read_csv('train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [26]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 5))),
                 ('tfidf', TfidfTransformer()),
                 #https://scikit-learn.org/stable/modules/svm.html
                ('clf', CalibratedClassifierCV(LinearSVC()))])# max iterations >2000 (max_iter=5000) for training prevents it from getting stuck for a long time on a small dataset

text_clf = text_clf.fit(df_train.text, df_train.target)

with open('svm_1_5_swin_classifier.pkl','wb') as f:
    pickle.dump(text_clf,f)

In [28]:
train = text_clf.predict_proba(df_train.text)
train = [0 if i[0]>0.5 else 1 for i in train]
test = text_clf.predict_proba(df_test.text)
test = [0 if i[0]>0.5 else 1 for i in test]
pd.DataFrame({'id':df_test.id.tolist(), 'target':test}).to_csv('submission.csv', index=False)

In [29]:
tp = sum([i for i,j in zip(train, df_train.target.tolist()) if ((i==1) and (j==1))])
fp = sum([i for i,j in zip(train, df_train.target.tolist()) if ((i==1) and (j==0))])
fn = sum([i for i,j in zip(train, df_train.target.tolist()) if ((i==0) and (j==1))])
precision = tp/(tp+fp)
recall =  tp/(tp+fn)
print(precision, recall)

0.9932473910374463 1.0


In [17]:
print('Done.')

Done.
